In [2]:
import torch as t
import torch.nn.functional as F
from tqdm import tqdm
from models.Autoencoder import Autoencoder
from torchvision import datasets, transforms

In [3]:
import os
os.environ['CUDA_VISIBLE_DIVICE']='0,1'

In [4]:
t.cuda.device_count()

2

In [5]:
t.cuda.get_device_name('cuda:0')

'GeForce RTX 3080 Ti'

In [6]:
t.cuda.get_device_capability('cuda:1')

(8, 6)

In [7]:
de = t.device('cuda:1')

In [8]:
t.randn(2,2).to(de)

tensor([[-0.0804, -0.4658],
        [-0.1049,  0.4448]], device='cuda:1')

In [9]:
t.randn(2,2).cuda(de)

tensor([[ 0.3966,  1.3238],
        [-0.0283,  1.2001]], device='cuda:1')

In [10]:
# t.cuda.empty_cache()

In [11]:
bs = 512
train_ds = datasets.MNIST(root='/home/new1006/ljh/data', train=True, transform=transforms.ToTensor(), download=True)
train_dl = t.utils.data.DataLoader(dataset=train_ds, batch_size=bs, shuffle=True, drop_last=True)

In [12]:
train_ds

Dataset MNIST
    Number of datapoints: 60000
    Root location: /home/new1006/ljh/data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [13]:
train_dl

In [14]:
for i in range(10):
    print(train_ds[i][0].shape, train_ds[i][1])

torch.Size([1, 28, 28]) 5
torch.Size([1, 28, 28]) 0
torch.Size([1, 28, 28]) 4
torch.Size([1, 28, 28]) 1
torch.Size([1, 28, 28]) 9
torch.Size([1, 28, 28]) 2
torch.Size([1, 28, 28]) 1
torch.Size([1, 28, 28]) 3
torch.Size([1, 28, 28]) 1
torch.Size([1, 28, 28]) 4


In [15]:
train_ds[0][0].size()

torch.Size([1, 28, 28])

In [16]:
train_ds[0][1]

5

In [17]:
train_ds[0][0][None].shape

torch.Size([1, 1, 28, 28])

In [18]:
train_ds[0][0].unsqueeze(0).shape

torch.Size([1, 1, 28, 28])

In [19]:
# train_ds[0][0][None] == train_ds[0][0].unsqueeze(0)

In [20]:
device = t.device('cuda:0') if t.cuda.is_available() else 'cpu'
model = Autoencoder(train_ds[0][0][None], in_c=1, enc_out_c=[32, 64, 64, 64],
                    enc_ks=[3, 3, 3, 3], enc_pads=[1, 1, 0, 1], enc_strides=[1, 2, 2, 1],
                    dec_out_c=[64, 64, 32, 1], dec_ks=[3, 3, 3, 3], dec_strides=[1, 2, 2, 1],
                    dec_pads=[1, 0, 1, 1], dec_op_pads=[0, 1, 1, 0], z_dim=2)
# model = Autoencoder(train_ds[0][0][None], in_c=1, enc_out_c=[32, 64, 64, 64],
#                     enc_ks=[3, 3, 3, 3], enc_pads=[1, 1, 1, 1], enc_strides=[1, 2, 2, 1],
#                     dec_out_c=[64, 64, 32, 1], dec_ks=[3, 3, 3, 3], dec_strides=[1, 2, 2, 1],
#                     dec_pads=[1, 1, 1, 1], dec_op_pads=[0, 1, 1, 0], z_dim=2)
model.cuda(device)

Autoencoder(
  (enc_conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=

In [21]:
device

device(type='cuda', index=0)

In [22]:
from torchsummary import summary

In [23]:
summary(model, input_size=(1, 28, 28), batch_size=512)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [512, 32, 28, 28]             320
         LeakyReLU-2          [512, 32, 28, 28]               0
       BatchNorm2d-3          [512, 32, 28, 28]              64
           Dropout-4          [512, 32, 28, 28]               0
            Conv2d-5          [512, 64, 14, 14]          18,496
         LeakyReLU-6          [512, 64, 14, 14]               0
       BatchNorm2d-7          [512, 64, 14, 14]             128
           Dropout-8          [512, 64, 14, 14]               0
            Conv2d-9            [512, 64, 6, 6]          36,928
        LeakyReLU-10            [512, 64, 6, 6]               0
      BatchNorm2d-11            [512, 64, 6, 6]             128
          Dropout-12            [512, 64, 6, 6]               0
           Conv2d-13            [512, 64, 6, 6]          36,928
        LeakyReLU-14            [512, 6

In [24]:
model

Autoencoder(
  (enc_conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=

In [25]:
for name, p in model.named_parameters():
    print(name, p.size())

enc_conv_layers.0.0.weight torch.Size([32, 1, 3, 3])
enc_conv_layers.0.0.bias torch.Size([32])
enc_conv_layers.0.2.weight torch.Size([32])
enc_conv_layers.0.2.bias torch.Size([32])
enc_conv_layers.1.0.weight torch.Size([64, 32, 3, 3])
enc_conv_layers.1.0.bias torch.Size([64])
enc_conv_layers.1.2.weight torch.Size([64])
enc_conv_layers.1.2.bias torch.Size([64])
enc_conv_layers.2.0.weight torch.Size([64, 64, 3, 3])
enc_conv_layers.2.0.bias torch.Size([64])
enc_conv_layers.2.2.weight torch.Size([64])
enc_conv_layers.2.2.bias torch.Size([64])
enc_conv_layers.3.0.weight torch.Size([64, 64, 3, 3])
enc_conv_layers.3.0.bias torch.Size([64])
enc_conv_layers.3.2.weight torch.Size([64])
enc_conv_layers.3.2.bias torch.Size([64])
enc_conv_layers.5.weight torch.Size([2, 2304])
enc_conv_layers.5.bias torch.Size([2])
dec_conv_layers.0.weight torch.Size([2304, 2])
dec_conv_layers.0.bias torch.Size([2304])
dec_conv_layers.2.0.weight torch.Size([64, 64, 3, 3])
dec_conv_layers.2.0.bias torch.Size([64])
de

In [26]:
import numpy as np

In [27]:
np.prod([1,2,3])

6

In [12]:
optimizer = t.optim.Adam(model.parameters(), lr=5e-4, betas=(.9, .99), weight_decay=1e-2)
model.train()

for epoch in tqdm(range(20)):
    if epoch == 10:
        optimizer = t.optim.Adam(model.parameters(), lr=2e-4, betas=(.9, .99), weight_decay=1e-2)
    for i, (data, _) in enumerate(train_dl):
        data = data.to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = F.mse_loss(pred, data)
        loss.backward()
        optimizer.step()
        if i % 33 == 0:
            print(loss)


  0%|                                                    | 0/20 [00:00<?, ?it/s]

tensor(0.3293, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.1019, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0823, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0699, device='cuda:0', grad_fn=<MseLossBackward0>)


  5%|██▏                                         | 1/20 [00:10<03:18, 10.46s/it]

tensor(0.0663, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0609, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0574, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0550, device='cuda:0', grad_fn=<MseLossBackward0>)


 10%|████▍                                       | 2/20 [00:19<02:55,  9.77s/it]

...

 90%|██████████████████████████████████████▋    | 18/20 [02:45<00:18,  9.14s/it]

tensor(0.0447, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0461, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0466, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0460, device='cuda:0', grad_fn=<MseLossBackward0>)


 95%|████████████████████████████████████████▊  | 19/20 [02:54<00:09,  9.11s/it]

tensor(0.0456, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0451, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0442, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0460, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|███████████████████████████████████████████| 20/20 [03:04<00:00,  9.23s/it]


In [14]:
print(loss)
t.save(model.state_dict(), 'models/state_dicts/03_01.pth')

tensor(0.0451, device='cuda:0', grad_fn=<MseLossBackward0>)


In [15]:
model.state_dict

<bound method Module.state_dict of Autoencoder(
  (enc_conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25, inplace=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 

In [16]:
model._modules

OrderedDict([('enc_conv_layers',
              Sequential(
                (0): Sequential(
                  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  (1): LeakyReLU(negative_slope=0.01)
                  (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (3): Dropout(p=0.25, inplace=False)
                )
                (1): Sequential(
                  (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
                  (1): LeakyReLU(negative_slope=0.01)
                  (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (3): Dropout(p=0.25, inplace=False)
                )
                (2): Sequential(
                  (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
                  (1): LeakyReLU(negative_slope=0.01)
                  (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [61]:
model.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('enc_conv_layers',
               Sequential(
                 (0): Sequential(
                   (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                   (1): LeakyReLU(negative_slope=0.01)
                   (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                   (3): Dropout(p=0.25, inplace=False)
                 )
                 (1): Sequential(
                   (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
                   (1): LeakyReLU(negative_slope=0.01)
                   (2): BatchNorm2d(64, eps=1e-05, m